<img  src="https://www.bcm.edu/themes/custom/bcm_bootstrap_subtheme/css/../images/BCM-90x90.svg" alt="Drawing" style="height: 100px;float: left;"/>


# Mask or no mask: A decision guide for teachers, parents, and students

* August, 2021
* Institute for Clinical & Translational Research, Baylor College of Medicine

# Context

With  increasing incidences of [COVID-19 infections in the Greater Houston Area](https://www.tmc.edu/coronavirus-updates/daily-new-covid-19-positive-cases-for-the-greater-houston-area/) and  students returning to school, informed decisions must be made regarding teacher and student continuous masking recommendations.

As of today (August 21),
1. An executive order is in place which [prohibits Texas government funded entities from mandating mask wearing on premises](https://gov.texas.gov/news/post/governor-abbott-issues-executive-order-prohibiting-government-entities-from-mandating-masks).
2. [Houston ISD](https://www.houstonisd.org/) is one of [a handful of schools in the Houston area](https://www.khou.com/article/news/health/coronavirus/masks-houston-school-districts-covid-coronavirus/285-25154fcd-97ae-4d8b-9444-be77a0b58f89) which does have a mask wearing mandate in place.
3. Colleges in the our area have established differing policies. Notably, Rice University has [COVID policies that mandate mask wearing indoors](https://coronavirus.rice.edu/policies). 

To provide quantitative insight into these mixed stances, we ran a targeted simulation using our [COVID-19 Outbreak Simulator](https://ictr.github.io/covid19-outbreak-simulator/) to compare the number of COVID-cases that develop in schools that mandate or do not mandate mask wearing.

**Disclaimer**: This report was prepared by researchers in the [Institute for Clinical & Translational Research](https://www.bcm.edu/research/research-offices/institute-for-clinical-translational-research), Baylor College of Medicine. It uses a limited model to predict the impact of mask wearing on COVID-19 incidences in public schools in the Greater Houston Area, and does not necessarily reflect the impact of other site specific COVID-prevention policies that may be in place at each of these schools.

# Conclusions

The delta variant is highly infectious. With the Houston's existing high regional community infection rate we found that:

1. Almost all schools will have at least 1 school-acquired infection.
2. If masks are worn at all the time, `0.8%` of high school students will become infected at school. This risk is, however, lower than getting infected from the community (`2.8%`). The proportions for school and community-based infections are higher for elementary schools(`1.0%` and `3.0%` respectively) because students in elementary schools are not vaccinated.
3. If masks are not worn at all, `15.4%` of high school students will be infected within a month, and one fifth (`21.0%`) all students in elementary schools will be infected. Again, we are assuming that there will be no post-infection contact tracing and testing will be performed so asymptomatic carriers will continue to attend school and infect others.
4. The proportion of students who will be isolated due to the showing of symptoms will be `1.8%` for high school students and `2.1%` for elementary school students if masks are worn. About `7.7%` of students in high school and `9.7%` of students in elementary schools will be isolated if masks are not worn.
5. If community infection rate drops to 1/4 of the current level (around `500` daily new cases in Harris County or `750` in the Greater Houston Area), students who will be isolated due to COVID-19 infections will be much lower, which will be around `0.5%` if masks are worn at all time, and `2.5%` if masks are not worn.
6. If we increase the vaccination coverage to an almost ideal level at which  `90%` of all teachers are vaccinated and `60%` of high school students are vaccinated, there will still be a significant number of breakthrough cases.

These results reflect our assumptions that

1. `60%` of teachers are vaccinated, `30%` of high school students are vaccinated, and no students in elementary schools are vaccinated.
2. Students in high schools have more physical interactions whereas students in elementary schools mostly interact with their own classmates.
3. Around `20%` of all students and teachers have already been infected and recovered from COVID.

Please see the `Methods` section for details.

# Results

We ran `10,000` replicate simulations for each scenario and the following table lists 

1. **Community infection rate (per mission)**, which is assumed to be `1600/million` now, but we also provide results for `400/million` for comparison purposes.
2. **School** can be either high school or elementary school, which differ by school size and student movement patterns.
3. **Vaccination Coverage (teachers / students)**. In addition to the assumed vaccination rates at the present time, we also simulated scenarios when `90%` of teachers are vaccinated and `60%` of high school students are vaccinated.
4. **Mask wearing**, can be no mask, with mask, or non-mandatory, for which assume 50% of students and teachers will wear mask.
5. **Percentage of schools that have no school-transmitted infection**. Students may be infected with the virus from home, but not from school.
6. **Percentage of students infected in community**. This number reflects risks from the community infection rate, vaccine coverage, and the proportion of students who have recovered from COVID infections.
7. **Percentage of students infected from school**. This number reflects the risk of the schools from which students get infected with the coronavirus.
8. **Percentage of students isolated due to COVID symptoms**. This number reflects the overall students who are impacted, including students who have been infected from both school and community, but exclude students who do not show any COVID symptoms.

In [37]:
## This function retrieves results for all contexts as a Python DataFrame.

import pickle
import os
import pandas as pd

def get_results(contexts):
    all_res = []
    for context in contexts:
        res_file = path(f'{context["name"]}.pickle')
        if not res_file.exists():
            print(f'Missing {context["name"]}')
            continue
        with open(res_file, 'rb') as infile:
            loaded = pickle.load(infile)
            #if not 'num_replicates' in loaded or loaded["num_replicates"] < context["num_replicates"]:
            #    print(f'WRONG {context["name"]}')
                #os.remove(res_file)
                #continue
            res = {
                'Community Infection Rate (per million)': int(context['community_infection_rate'] * 1000000),
                'School': context['school'].split('_')[-1],
                'Vaccination Coverage (teachers / students)': context['vac_coverage_text'],
                'Mask wearing': context['mask_wearing'],
#                 'Vaccination coverage (%)': int(context['vac_coverage'] * 100),
                
            }
            res.update(loaded)
            res.pop('num_replicates', None)
            all_res.append(res)

    all_res = pd.DataFrame(all_res)
    return all_res

pd.options.display.float_format = '{:,.1f}'.format


In [43]:
%preview  res 

res = get_results(all_contexts)

with pd.ExcelWriter(path(f'results.xlsx'), engine='xlsxwriter') as writer: 
    res.to_excel(writer, index=False, sheet_name='All')
    
    workbook  = writer.book
    
    fmt_num = workbook.add_format({'num_format': '0.00'})
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True})
    
    worksheet = writer.sheets["All"]
    worksheet.set_column('J:P', None, fmt_num)
    #worksheet.set_row(0, 0, header_format)

>>> res:

,Community Infection Rate (per million),School,Vaccination Coverage (teachers / students),Mask wearing,Percentage of schools with no school-transmitted infections %,"Percentage of students infected in community, with 95% CI","Percentage of students infected at school, with 95% CI","Percentage of students isolated due to COVID symptoms, with 95% CI"
0,1600,high,60% / 30%,with_mask,0.8,"2.8 [1.8, 4.0]","0.8 [0.1, 1.8]","1.9 [1.0, 3.0]"
1,1600,high,60% / 30%,partial_mask,0.0,"2.8 [1.8, 3.9]","4.2 [1.4, 8.4]","3.5 [1.6, 6.1]"
2,1600,high,60% / 30%,no_mask,0.0,"2.7 [1.6, 3.8]","15.4 [5.0, 29.1]","7.7 [3.0, 14.2]"
3,1600,elementary,60% / 0%,with_mask,2.5,"3.0 [1.6, 4.6]","1.0 [0.0, 2.4]","2.1 [0.8, 3.8]"
4,1600,elementary,60% / 0%,partial_mask,0.0,"2.9 [1.6, 4.4]","5.8 [1.2, 12.6]","4.2 [1.4, 8.2]"
5,1600,elementary,60% / 0%,no_mask,0.0,"2.8 [1.6, 4.2]","21.2 [5.2, 41.8]","9.8 [2.6, 19.6]"
6,1600,high,90% / 60%,with_mask,4.0,"2.5 [1.5, 3.6]","0.5 [0.0, 1.2]","1.6 [0.8, 2.6]"
7,1600,high,90% / 60%,partial_mask,0.0,"2.5 [1.5, 3.6]","2.2 [0.6, 4.8]","2.5 [1.1, 4.2]"
8,1600,high,90% / 60%,no_mask,0.0,"2.4 [1.5, 3.5]","6.4 [1.9, 13.0]","4.2 [1.6, 7.8]"
9,1600,elementary,90% / 0%,with_mask,3.0,"3.0 [1.6, 4.4]","1.0 [0.0, 2.4]","2.1 [0.8, 3.8]"


# Methods

We assume that all students and teachers live in the same "community" that is subject to a fixed "community infection rate", which is the probability that people are infected. People who are infected from the community would go to school, with or without mask, and have the same probability of infecting others. People who show symptoms are isolated for 10 days before they go back to school, at which time they have recovered and are largely immune to reinfection. We **do not yet model other post-infection reactions such as contact tracing and testing**, which are being employed by many schools and universities to prevent the spread of the coronavirus.

We assume 

* **Community infection rate (CIR)**: CIR is the actual community rate of infection and is used to model the probability at which people are infected. The actual number will differ from the Public Health reported confirmed case numbers due to reporting delays and the sufficiency of testing (see https://ourworldindata.org/covid-models for an expanded discussion). As of writing, 
    * There are around `2000` confirmed cases per day in the Harris County with `4.7 M` people, or
    * Around `3000` confirmed cases per day in the Greater Houston Area with a population of about `9` million,
    
  so the raw CIR is about `400 / million`. Using a popular method to adjust for the under-reporting of actual CIR, namely multiplying the number of confirmed cases by a factor of 4, we use **1600 / million** as the CIR for the Greater Houston Area. Note that the estimates from [The Institute for Health Metrics and Evaluation (IHME) an independent global health research center at the University of Washington](https://covid19.healthdata.org/global?view=cumulative-deaths&tab=trend) is around 2121 per million for the state of Texas.

* **Vaccination coverage**: According to government report, 45.7% of people are fully vaccinated in Texas. We assume that adults (teachers) have higher vaccination rates and assume that
    * `60%` of teachers and staff are vaccinated in both high schools and elementary schools.
    * `30%` of students are vaccinated in high schools.
    * Students in elementary schools are not vaccinated.
    * For comparison purposes, we also simulated scenarios under which `90%` of teachers are vaccinated and `60%` of high school students are vaccinated.
  
* **Size of schools**: According to online information, we assume that
    * High schools have 800 students with a 15:1 teacher student ratio (50 teachers, 50 staff).
    * Elementary schools have 500 students with a 15:1 teacher student ratio (30 teachers, 30 staff).
    
* **Transmissibility of virus**: The highly contagious delta variant has become the most prevalent version of COVID-19 in the country, and also in the state of Texas. The delta variant has 
    * A reproduction number of around `6.5` for symptomatic cases (range 5-8)
    * A reproduction number of `4.6` for asymptomatic carriers because we assume that asymptomatic carriers are 70% infectious compared to symptomatic carriers.
    * An average incubation period of `4.4`
    * `30%` of people will remain asymptomatic after being infected.
    * `20%` of the population (teachers and students) have already been infected, and recovered. They are less susceptible to another infection (`50%` reduction) and have lower viral load (`50%` reduction) if they are infected.    
    
* **Infection patterns**:
    * We assume that students and teachers will spend half of the "interacting" time at home and half of the time at school (8 hours each), so only about half of the infection events will happen at school. 
    * We assume that students in elementary schools are divided into `25` classes with `20` students each. Because students in elementary schools usually only interact with their classmates and teachers, we assume that each student will interact with the rest of the class (`19` students), `3` teachers, and `5` random students they interact at hall way or cafeterias.
    * We assume that students in high schools are well mixed so everyone can infect any other student, teacher, or staff.

* **Post-infection reaction**:
    * We assume that no testing will be performed and students will only be sent home and isolate for 10 days after they have shown symptoms of COVID-19.
    * Teachers and students will be allowed to go back to school after 10 days. They are assumed to be recovered, but can be infected again.
    
* **Efficacy of vaccine**: Most Houstonians were vaccinated with the Pfizer vaccine, which is reported to
    * Has `42%` efficacy for preventing infection
    * Reduces viral loads of infected people by around `50%`
    * Has around `50%` efficacy for preventing secondary infection.
 
* **Duration of simulation**: We simulate the operation of schools for a month (28 days, 20 working days).
 
* **Mask wearing**: We simulate three scenarios,
   * no mask
   * with mask, which causes 80% reduction in the transmission of the virus
   * some mask, which corresponds to the case of no mandated mask wearing, perhaps 50% of people will wear mask and lead to 40% reduction in the transmission of the virus.
 

We output the number of infections and symptomatic cases in both high schools and elementary schools.


In [41]:
import pandas as pd

school_spec = {
    'high': {'teachers': 'T=100', 'students': 'S=800', 'vac_coverage': 'T=0.6 S=0.3', 'vac_coverage_text': '60% / 30%' },
    'elementary': {'teachers': 'T=60', 
                   'students': ' '.join([f'S{i+1}=20' for i in range(25)]),
                   'vac_coverage': 'T=0.6 ' + ' '.join([f'S{i+1}=0' for i in range(25)]),
                   'vac_coverage_text': '60% / 0%' },
    #
    'vac_high': {'teachers': 'T=100', 'students': 'S=800', 'vac_coverage': 'T=0.9 S=0.6', 'vac_coverage_text': '90% / 60%'  },
    'vac_elementary': {'teachers': 'T=60', 
                   'students': ' '.join([f'S{i+1}=20' for i in range(25)]),
                   'vac_coverage': 'T=0.9 ' + ' '.join([f'S{i+1}=0' for i in range(25)]), 'vac_coverage_text': '90% / 0%'  }
}

communities = {
    'C1600': dict(community_infection_rate=1600 / 1000000),
    'C400': dict(community_infection_rate=400 / 1000000),
}


variants = {
    'delta': dict(sym_r0=[5, 8], asym_r0=[5*0.75, 8*0.75], incu_period=4.4, name='B.1.617.2', prop_asym_carriers=0.3),
}

# 86/76 for alpha; 76/42 for delta; and 75-81% protection against hospitalizations.
vaccination_specs = {
    'pfizer': dict(
            delta=dict(vac_immunity=[0.42, 0.38], vac_infectivity=[0.5, 0.5]),
        ),    
}

distancing_spec = {
    'no_mask': 1,
    'with_mask': 0.2,
    'partial_mask': 0.6
}

def get_single_context(
    school,
    distancing,
    community,
):
    variant = 'delta'
    vac_name = 'pfizer'
    return dict(
        name=f'{community}_{school}_{distancing}',
        school=school,
        mask_wearing=distancing,
        community=community,
        #
        community_infection_rate=communities[community]['community_infection_rate'],
        #
        vac_proportion=school_spec[school]['vac_coverage'],
        vac_coverage_text=school_spec[school]['vac_coverage_text'],
        
        vac_immunity=' '.join(str(x) for x in vaccination_specs[vac_name][variant]['vac_immunity']),
        vac_infectivity=' '.join(str(x) for x in vaccination_specs[vac_name][variant]['vac_infectivity']),
                
        # constant
        pop_size=school_spec[school]['teachers'] + ' ' + school_spec[school]['students'],
        prop_asym_carriers=variants[variant]["prop_asym_carriers"],
        
        sym_r0=f'{variants[variant]["sym_r0"][0]:.2f} {variants[variant]["sym_r0"][1]:.2f}',
        asym_r0=f'{variants[variant]["asym_r0"][0]:.2f} {variants[variant]["asym_r0"][1]:.2f}',
        incu_period=variants[variant]["incu_period"],        
        # use 0.5 to assume that half of the infections will be to
        # non-school members
        distancing_multiplier=distancing_spec[distancing] * 0.5,
        duration=20,
        immunity_of_recovered=0.5,
        infectivity_of_recovered=0.5,  
        prop_recovered = 0.2,
        #
        # use more replicates for lower community infection rate to save computing time
        num_replicates=10000)


def unique_contexts(contexts):
    names = set()
    res = []
    for context in contexts:
        if not context:
            continue
        if context['name'] in names:
            continue
        res.append(context)
        names.add(context['name'])
    return res


def get_contexts(
    school_cases=[],    
    distancing_cases=[],    
    community_cases=[],
):
    contexts = []
    for community in community_cases:
        for school in school_cases:
            for distancing in distancing_cases:            
                contexts.append(
                                get_single_context(school, distancing, community))
    # remove duplicate
    return unique_contexts(contexts)


In [42]:
%preview cases -l 400

high_contexts = get_contexts(
    school_cases=['high', 'vac_high'],
    community_cases=[
        'C1600', 'C400'
    ],
    distancing_cases=['with_mask', 'partial_mask', 'no_mask'],
)

elementary_contexts = get_contexts(
    school_cases=['elementary', 'vac_elementary'],
    community_cases=[
        'C1600', 'C400'
    ],
    distancing_cases=['with_mask', 'partial_mask', 'no_mask'],
)

all_contexts = get_contexts(
    school_cases=['high', 'elementary', 'vac_high', 'vac_elementary'],
    community_cases=[
        'C1600', 'C400'
    ],
    distancing_cases=['with_mask', 'partial_mask', 'no_mask'],
)


cases = pd.DataFrame(all_contexts)

>>> cases:

,name,school,mask_wearing,community,community_infection_rate,vac_proportion,vac_coverage_text,vac_immunity,vac_infectivity,pop_size,prop_asym_carriers,sym_r0,asym_r0,incu_period,distancing_multiplier,duration,immunity_of_recovered,infectivity_of_recovered,prop_recovered,num_replicates
0,C1600_high_with_mask,high,with_mask,C1600,0.0,T=0.6 S=0.3,60% / 30%,0.42 0.38,0.5 0.5,T=100 S=800,0.3,5.00 8.00,3.75 6.00,4.4,0.1,20,0.5,0.5,0.2,10000
1,C1600_high_partial_mask,high,partial_mask,C1600,0.0,T=0.6 S=0.3,60% / 30%,0.42 0.38,0.5 0.5,T=100 S=800,0.3,5.00 8.00,3.75 6.00,4.4,0.3,20,0.5,0.5,0.2,10000
2,C1600_high_no_mask,high,no_mask,C1600,0.0,T=0.6 S=0.3,60% / 30%,0.42 0.38,0.5 0.5,T=100 S=800,0.3,5.00 8.00,3.75 6.00,4.4,0.5,20,0.5,0.5,0.2,10000
3,C1600_elementary_with_mask,elementary,with_mask,C1600,0.0,T=0.6 S1=0 S2=0 S3=0 S4=0 S5=0 S6=0 S7=0 S8=0 S9=0 S10=0 S11=0 S12=0 S13=0 S14=0 S15=0 S16=0 S17=0 S18=0 S19=0 S20=0 S21=0 S22=0 S23=0 S24=0 S25=0,60% / 0%,0.42 0.38,0.5 0.5,T=60 S1=20 S2=20 S3=20 S4=20 S5=20 S6=20 S7=20 S8=20 S9=20 S10=20 S11=20 S12=20 S13=20 S14=20 S15=20 S16=20 S17=20 S18=20 S19=20 S20=20 S21=20 S22=20 S23=20 S24=20 S25=20,0.3,5.00 8.00,3.75 6.00,4.4,0.1,20,0.5,0.5,0.2,10000
4,C1600_elementary_partial_mask,elementary,partial_mask,C1600,0.0,T=0.6 S1=0 S2=0 S3=0 S4=0 S5=0 S6=0 S7=0 S8=0 S9=0 S10=0 S11=0 S12=0 S13=0 S14=0 S15=0 S16=0 S17=0 S18=0 S19=0 S20=0 S21=0 S22=0 S23=0 S24=0 S25=0,60% / 0%,0.42 0.38,0.5 0.5,T=60 S1=20 S2=20 S3=20 S4=20 S5=20 S6=20 S7=20 S8=20 S9=20 S10=20 S11=20 S12=20 S13=20 S14=20 S15=20 S16=20 S17=20 S18=20 S19=20 S20=20 S21=20 S22=20 S23=20 S24=20 S25=20,0.3,5.00 8.00,3.75 6.00,4.4,0.3,20,0.5,0.5,0.2,10000
5,C1600_elementary_no_mask,elementary,no_mask,C1600,0.0,T=0.6 S1=0 S2=0 S3=0 S4=0 S5=0 S6=0 S7=0 S8=0 S9=0 S10=0 S11=0 S12=0 S13=0 S14=0 S15=0 S16=0 S17=0 S18=0 S19=0 S20=0 S21=0 S22=0 S23=0 S24=0 S25=0,60% / 0%,0.42 0.38,0.5 0.5,T=60 S1=20 S2=20 S3=20 S4=20 S5=20 S6=20 S7=20 S8=20 S9=20 S10=20 S11=20 S12=20 S13=20 S14=20 S15=20 S16=20 S17=20 S18=20 S19=20 S20=20 S21=20 S22=20 S23=20 S24=20 S25=20,0.3,5.00 8.00,3.75 6.00,4.4,0.5,20,0.5,0.5,0.2,10000
6,C1600_vac_high_with_mask,vac_high,with_mask,C1600,0.0,T=0.9 S=0.6,90% / 60%,0.42 0.38,0.5 0.5,T=100 S=800,0.3,5.00 8.00,3.75 6.00,4.4,0.1,20,0.5,0.5,0.2,10000
7,C1600_vac_high_partial_mask,vac_high,partial_mask,C1600,0.0,T=0.9 S=0.6,90% / 60%,0.42 0.38,0.5 0.5,T=100 S=800,0.3,5.00 8.00,3.75 6.00,4.4,0.3,20,0.5,0.5,0.2,10000
8,C1600_vac_high_no_mask,vac_high,no_mask,C1600,0.0,T=0.9 S=0.6,90% / 60%,0.42 0.38,0.5 0.5,T=100 S=800,0.3,5.00 8.00,3.75 6.00,4.4,0.5,20,0.5,0.5,0.2,10000
9,C1600_vac_elementary_with_mask,vac_elementary,with_mask,C1600,0.0,T=0.9 S1=0 S2=0 S3=0 S4=0 S5=0 S6=0 S7=0 S8=0 S9=0 S10=0 S11=0 S12=0 S13=0 S14=0 S15=0 S16=0 S17=0 S18=0 S19=0 S20=0 S21=0 S22=0 S23=0 S24=0 S25=0,90% / 0%,0.42 0.38,0.5 0.5,T=60 S1=20 S2=20 S3=20 S4=20 S5=20 S6=20 S7=20 S8=20 S9=20 S10=20 S11=20 S12=20 S13=20 S14=20 S15=20 S16=20 S17=20 S18=20 S19=20 S20=20 S21=20 S22=20 S23=20 S24=20 S25=20,0.3,5.00 8.00,3.75 6.00,4.4,0.1,20,0.5,0.5,0.2,10000


In [16]:
input: for_each=high_contexts

task: queue='localhost', cores=4, walltime='1h', mem='4G', tags=name, workdir='.', trunk_size=1

sh: expand=True
  #rm -f {name}.log.lock
  outbreak_simulator --popsize {pop_size} \
      --stop-if 't>{duration}' -j 4 \
      --track-events INFECTION WARNING PLUGIN SHOW_SYMPTOM QUARANTINE \
      --repeat {num_replicates} --resume \
      --symptomatic-r0 {sym_r0} T={distancing_multiplier} S={distancing_multiplier} \
      --asymptomatic-r0 {asym_r0} T={distancing_multiplier} S={distancing_multiplier} \
      --immunity-of-recovered {immunity_of_recovered} \
      --infectivity-of-recovered {infectivity_of_recovered} \
      --incubation-period {incu_period} \
      --prop-asym-carriers {prop_asym_carriers} \
      --handle-symptomatic 'quarantine?duration=10&infected=true' \
      --logfile {name}.log \
      --plugin init \
          --incidence-rate {community_infection_rate} \
          --seroprevalence {prop_recovered} --as-proportion \
      --plugin vaccinate \
          --start 0 --proportion {vac_proportion} --immunity {vac_immunity} \
          --infectivity {vac_infectivity} \
      --plugin community_infection --start 0 --interval 1 --probability {community_infection_rate} 

INFO: t1f5d26e4fabbe33b re-execute completed

,t1f5d26e4fabbe33b,C1600_high_with_maskac8254c6575b9a47back-to-schoolcell_6fe6ebeb,Ran for < 5 seconds,completed


INFO: t8ddfd167f6578676 re-execute completed

,t8ddfd167f6578676,C1600_high_partial_maskac8254c6575b9a47back-to-schoolcell_6fe6ebeb,Ran for < 5 seconds,completed


INFO: tc4f80e59fea95dfd re-execute completed

...

,taa0d9dbbae9aa402,C400_high_partial_maskac8254c6575b9a47back-to-schoolcell_ccdbf19f,Ran for < 5 seconds,completed


,t7ae93cce53ca6f30,C400_high_no_maskac8254c6575b9a47back-to-schoolcell_ccdbf19f,Ran for < 5 seconds,completed


,ta891a05568d2c595,C400_vac_high_no_maskac8254c6575b9a47back-to-schoolcell_ccdbf19f,Ran for 4 min 38 sec,completed


,tf2c39a6a22f68500,C1600_vac_high_with_maskac8254c6575b9a47back-to-schoolcell_ccdbf19f,Ran for < 5 seconds,completed


,tb38f62b1aaab0b9c,C400_high_with_maskac8254c6575b9a47back-to-schoolcell_ccdbf19f,Ran for < 5 seconds,completed


In [22]:
input: for_each=elementary_contexts

task: queue='localhost', cores=4, walltime='2h', mem='4G', tags=name, workdir='.', trunk_size=1

sh: expand=True
  rm -f {name}.log.lock
  outbreak_simulator --popsize {pop_size} \
      --stop-if 't>{duration}' -j 4 \
      --track-events INFECTION WARNING PLUGIN SHOW_SYMPTOM QUARANTINE \
      --vicinity 'T-S*=0.5' 'T-T=10' 'S*-!&=0.25' 'S*-&=19' 'S*-T=3' \
      --repeat {num_replicates} \
      --symptomatic-r0 {sym_r0} T={distancing_multiplier} 'S*={distancing_multiplier}' \
      --asymptomatic-r0 {asym_r0} T={distancing_multiplier} 'S*={distancing_multiplier}' \
      --immunity-of-recovered {immunity_of_recovered} \
      --infectivity-of-recovered {infectivity_of_recovered} \
      --incubation-period {incu_period} \
      --prop-asym-carriers {prop_asym_carriers} \
      --handle-symptomatic 'quarantine?duration=10&infected=true' \
      --logfile {name}.log \
      --plugin init \
          --incidence-rate {community_infection_rate} \
          --seroprevalence {prop_recovered} --as-proportion \
      --plugin vaccinate \
          --start 0 --proportion {vac_proportion} --immunity {vac_immunity} \
          --infectivity {vac_infectivity} \
      --plugin community_infection --start 0 --interval 1 --probability {community_infection_rate} 

,t8cb191c41fedaab3,C1600_elementary_with_maskback-to-schoolcell_e16188ead46ad6e811bf38b6,Ran for 10 min 54 sec,completed


,te07a68dcdb0ca2b3,C1600_elementary_partial_maskback-to-schoolcell_e16188ead46ad6e811bf38b6,Ran for 12 min 6 sec,completed


,teef774042ef3e34b,C1600_elementary_no_maskback-to-schoolcell_e16188ead46ad6e811bf38b6,Ran for 15 min 51 sec,completed


,t82703cba10e88790,C1600_vac_elementary_with_maskback-to-schoolcell_e16188ead46ad6e811bf38b6,Ran for 11 min 32 sec,completed


,t8cbb2775a2f9fe7a,C1600_vac_elementary_partial_maskback-to-schoolcell_e16188ead46ad6e811bf38b6,Ran for 7 min 5 sec,completed


...

,taf2b98a545a5e2dc,C400_elementary_partial_maskback-to-schoolcell_e16188ead46ad6e811bf38b6,Ran for 5 min 59 sec,completed


,t97173f6492a1bf8a,C400_elementary_no_maskback-to-schoolcell_e16188ead46ad6e811bf38b6,Ran for 12 min 31 sec,completed


,tec9f5901dc4b17b9,C400_vac_elementary_with_maskback-to-schoolcell_e16188ead46ad6e811bf38b6,Ran for 10 min 59 sec,completed


,t9f2b100a8482f495,C400_vac_elementary_partial_maskback-to-schoolcell_e16188ead46ad6e811bf38b6,Ran for 11 min 22 sec,completed


,t55697109f964bde9,C400_vac_elementary_no_maskback-to-schoolcell_e16188ead46ad6e811bf38b6,Ran for 8 min 32 sec,completed


In [26]:
import pickle
import os
import re
import csv
import pandas as pd
from collections import defaultdict

def get_result(name):
    print(f'Processing {name}', flush=True)

    res = {}
    
    file = open(name + '.log', 'rU')
    reader = csv.reader(file, delimiter='\t')
    headers = next(reader, None)
    IDs = set()
    
    n_s_workplace_infection = defaultdict(int)
    n_t_workplace_infection = defaultdict(int)
    n_s_community_infection = defaultdict(int)
    n_t_community_infection = defaultdict(int)
    n_s_quarantine_due_to_symptom = defaultdict(int)
    n_t_quarantine_due_to_symptom = defaultdict(int)
    
    n_workplace_infections = defaultdict(int)
    n_community_infections = defaultdict(int)
    
    for row in reader:
        IDs.add(row[0])
        if row[2] == 'INFECTION':
            if row[4].startswith('by=.'):
                if row[3].startswith('S'):
                    n_s_community_infection[row[0]] += 1                    
                else:
                    n_t_community_infection[row[0]] += 1
                n_community_infections[row[0]] += 1
            else:
                if row[3].startswith('S'):
                    n_s_workplace_infection[row[0]] += 1                    
                else:
                    n_t_workplace_infection[row[0]] += 1
                n_workplace_infections[row[0]] += 1
        elif row[2] == 'QUARANTINE':
            if 'reason=show symptom' in row[4]:
                if row[3].startswith('S'):
                    n_s_quarantine_due_to_symptom[row[0]] += 1                    
                else:
                    n_t_quarantine_due_to_symptom[row[0]] += 1                      
    file.close()
    
    replicates = len(IDs)
                
    # we need to scale to 2000 person weeks
    # num_events = 8 weeks * 60 people * replicates
    # num_events * 2000/replicates/8/60 = 2000 week person
    school = "high" if 'high' in name else 'elementary'
    if school == "high":
        scaling_factor = 1 / replicates
        n_teachers = 100
        n_students = 800
    else:
        scaling_factor = 1 / replicates
        n_teachers = 60
        n_students = 500
        
    for ID in IDs:
        for d, n in [
            (n_s_workplace_infection, n_students),
            (n_t_workplace_infection, n_teachers),
            (n_s_community_infection, n_students),
            (n_t_community_infection, n_teachers),
            (n_s_quarantine_due_to_symptom, n_students),
            (n_t_quarantine_due_to_symptom,   n_teachers)
        ]:
            d[ID] = d[ID] / n

    def m_ci(d):
        m = sum(d.values()) / len(d)
        l = pd.Series(d.values()).quantile(0.025)
        h = pd.Series(d.values()).quantile(0.975)
        return f'{m*100:.1f} [{l*100:.1f}, {h*100:.1f}]'
    
        
    res['Percentage of schools with no school-transmitted infections %'] = 100 - len(n_workplace_infections) * scaling_factor * 100
    
    #res['Avg number of community acquired Infections'] = (n_s_community_infection + n_t_community_infection ) * scaling_factor    
    #res['Avg number of school acquired Infections'] = (n_s_workplace_infection + n_t_workplace_infection) * scaling_factor
    
    res['Percentage of students infected in community, with 95% CI'] = m_ci(n_s_community_infection)
    
    res['Percentage of students infected at school, with 95% CI'] = m_ci(n_s_workplace_infection)
    #res['Avg proportion of teacher infected from school %'] = (n_t_workplace_infection * scaling_factor) / n_teachers * 100
    
    res['Percentage of students isolated due to COVID symptoms, with 95% CI'] = m_ci(n_s_quarantine_due_to_symptom)
    
    #res['Avg proportion of symptomatic teachers %'] = n_t_quarantine_due_to_symptom * scaling_factor  / n_teachers * 100
    #res['num_replicates'] = replicates

    with open(name + '.pickle', 'wb') as outfile:
        pickle.dump(res, outfile)
    return res

In [28]:
for name in [x['name'] for x in all_contexts]:
    get_result(name)

Processing C1600_high_with_mask
Processing C1600_high_partial_mask
Processing C1600_high_no_mask
Processing C1600_elementary_with_mask
Processing C1600_elementary_partial_mask
Processing C1600_elementary_no_mask
Processing C1600_vac_high_with_mask
Processing C1600_vac_high_partial_mask
Processing C1600_vac_high_no_mask
Processing C1600_vac_elementary_with_mask
Processing C1600_vac_elementary_partial_mask
Processing C1600_vac_elementary_no_mask
Processing C400_high_with_mask
Processing C400_high_partial_mask
Processing C400_high_no_mask
Processing C400_elementary_with_mask
Processing C400_elementary_partial_mask
Processing C400_elementary_no_mask
Processing C400_vac_high_with_mask
Processing C400_vac_high_partial_mask
Processing C400_vac_high_no_mask
Processing C400_vac_elementary_with_mask
Processing C400_vac_elementary_partial_mask
Processing C400_vac_elementary_no_mask
